In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from tqdm import tqdm
from scipy import ndimage

In [2]:
import torch.nn.functional as F
import torch as tc
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import TensorDataset
from helpers_img import *
from NeuralNets import *
#from training_NN import *
from preprocessing import *

# try to improve simple_net predictions

# Define A NewNet

In [29]:
class DeepNet(tc.nn.Module):
    def __init__(self,dropout,features=3):
        super(DeepNet,self).__init__()
        self.conv1=tc.nn.Conv2d(features,16,kernel_size=(3,3))
        self.pool1= tc.nn.MaxPool2d(kernel_size = (2,2))
        self.drop1=tc.nn.Dropout(dropout)
        self.batch1=tc.nn.BatchNorm2d(16)
        self.conv2=tc.nn.Conv2d(16,16,kernel_size=(4,4))
        self.pool2 = tc.nn.MaxPool2d(kernel_size = (2,2))
        self.drop2=tc.nn.Dropout(dropout)
        self.batch12=tc.nn.BatchNorm2d(16)
        self.conv3=tc.nn.Conv2d(16,32,kernel_size=(3,3))
        self.pool3= tc.nn.MaxPool2d(kernel_size = (2,1))
        self.drop3=tc.nn.Dropout(dropout)
        self.batch12=tc.nn.BatchNorm2d(32)
        self.conv4=tc.nn.Conv2d(32,1,kernel_size=(2,2))
        self.pool4 = tc.nn.MaxPool2d(kernel_size = (2,1))
        self.drop4=tc.nn.Dropout(dropout)
        # Net created for input_files 16*16
        self.fc1=tc.nn.Linear(7,7)
        self.drop5 = tc.nn.Dropout(dropout)
        self.fc2=tc.nn.Linear(7,1)

    def forward(self,x):
        x = F.relu(self.pool1(self.conv1(x)))
        x = self.drop1(x)
        x = F.relu(self.pool2(self.conv2(x)))
        x = self.drop2(x)
        x = F.relu(self.pool3(self.conv3(x)))
        x = self.drop3(x)
        x = F.relu(self.pool4(self.conv4(x)))
        x = self.drop4(x)
        
        x = F.relu(self.fc1(x))
        x = self.drop5(x)
        x= tc.sigmoid(self.fc2(x))
        #x = self.fc2(x)
        return x
    

In [ ]:
? tc.nn.MaxPool2d

In [4]:
def img_crop_mod(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(h,imgheight-h,h):
        for j in range(w,imgwidth-w,w):
            if is_2d:
                im_patch = im[j-w:j+2*w, i-h:i+2*h]
            else:
                im_patch = im[j-w:j+2*w, i-h:i+2*h, :]
            list_patches.append(im_patch)
    return list_patches

In [27]:
class DatasetDeepNet(Dataset):
    def __init__(self,root_dir, do_flip=False, do_rotation=False,do_train=False):
        self.image_dir = root_dir + "images/"
        self.files = os.listdir(self.image_dir)
        self.gt_dir = root_dir + "groundtruth/"
        self.rot_len=0
        self.flip_len=0
        self.train = do_train
        self.initial_len=len(self.files)
        # rotation
        if do_rotation:
            self.rot_len= len(self.files)
            self.files = [*self.files*4]
        #flip 
        if do_flip:
            self.flip_len=len(self.files)
            self.files= [*self.files*2]
        
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self,index):
        image = [load_image(self.image_dir + self.files[index])]
        gt_image = [load_image(self.gt_dir + self.files[index])]
        if self.rot_len>0:
            image,gt_image = rotation(image,gt_image)
        if self.flip_len>0:
            image,gt_image = flip(image,gt_image)
        
        i = index//self.initial_len
        image,gt_image = image[i],gt_image[i]
        image = add_border(image,432)
        train_sub_images = [img_crop_mod(image, 16, 16)]
        train_mask_label = [img_crop(gt_image,16,16)]
        train_mask_label = from_mask_to_vector(train_mask_label,0.3)
        train_sub_images = transform_subIMG_to_Tensor(train_sub_images)
        mean = train_sub_images.mean()
        std = train_sub_images.std()
        train_sub_images = (train_sub_images-mean)/std
        if self.train:
            train_sub_images, train_mask_label = reduce_dataset(train_sub_images,train_mask_label)
            for l in range(10):
                new_indices= np.random.permutation(len(train_mask_label))
                train_sub_images=train_sub_images[new_indices]
                train_mask_label=train_mask_label[new_indices]
            
        return train_sub_images, train_mask_label

In [42]:
def train_model_Adam_v2( model, dataset, max_epochs, lr, mini_batch_size, w=48, h=48, features=3, threshold=0.01):
    '''train the Neural Net using Adam as optimizer and an binary cross entropy loss'''
    #optimizer=tc.optim.SGD(model.parameters(),lr)
    train_loader = DataLoader(dataset,batch_size=mini_batch_size)
    optimizer=tc.optim.Adam(model.parameters(),lr)
    criterion= tc.nn.BCELoss()
    losses=[]
    training_errors = []
    if tc.cuda.is_available():
        model.cuda()
        criterion.cuda()
    
    for epoch in tqdm(range(max_epochs)):
        model.is_training=True
        model.train()
    
        for train_data,label in train_loader:
            train_data = train_data.view(-1,features,w,h)
            label = label.view(-1,1).type(tc.FloatTensor)
            if tc.cuda.is_available():
                train_data = train_data.cuda()
                label = label.cuda()
            output= model(train_data).view(-1,1)
            #print(output,tc.LongTensor(np.array([1*label[i:i+mini_batch_size]]).reshape(-1,1)))
            loss= criterion(output,label)
            model.zero_grad()
            loss.backward()
            optimizer.step()
        losses.append(loss)
        # compute training error
        '''
        model.eval()
        test = model(train_data)
        test = test.cpu()
        prediction= test[:]>0.5
        
        prediction= 1*(prediction.numpy()[:] != label.reshape(-1,1)[:])
        
        training_error = np.sum(prediction)/len(prediction)
        training_errors.append(training_error*100)
        if training_error< threshold:
            break
        
    plt.figure()
    plt.plot(np.arange(epoch+1)+1,training_errors)
    plt.xlabel('epoch')
    plt.ylabel('training_errors')
    plt.show()
    '''
    plt.figure()
    plt.plot(np.arange(epoch+1)+1,losses)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()
    
    model.cpu()    
    return model

In [15]:
def trainDeepNet(root_dir, max_epochs, lr, mini_batch_size, dropout=0, model = None):
    if model == None:
        model = DeepNet(dropout)
    
    dataset = DatasetDeepNet(root_dir, do_flip=True, do_rotation=True,do_train=True)
    
    model = train_model_Adam_v2( model, dataset, max_epochs, lr, mini_batch_size)
    return model

In [ ]:
 root_dir = 'training/'
max_epochs = 15
lr = 1e-4
mini_batch_size = 1
model = trainDeepNet(root_dir, max_epochs, lr, mini_batch_size, model = None)


 20%|████████                                | 3/15 [23:34<1:34:06, 470.52s/it]